In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
torch.manual_seed(42)

transform = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [3]:
train_dataset = datasets.ImageFolder(root=r'C:\Guvi_final_project\Image Classification\dataset\training_set', transform=transform)
train_loader = Data.DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = datasets.ImageFolder(root=r'C:\Guvi_final_project\Image Classification\dataset\test_set', transform=transform)
test_loader = Data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)
        self.convres = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        y = self.pool(self.pool(self.pool(self.convres(x))))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x += y
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [6]:
import time

st=time.time()

train_acc0=[]
test_acc0=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc0.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc0.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')

    with open('Model3_test.pkl','wb') as file:
            pickle.dump(test_acc0,file)
    with open('Model3_train.pkl','wb') as file:
            pickle.dump(train_acc0,file)
    with open('Model3_test.pkl','rb') as file:
            test_acc0=pickle.load(file)
    with open('Model3_train.pkl','rb') as file:
            train_acc0=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')


Epoch 1/30, Step [500/2000], Loss: 0.4519
Epoch 1/30, Step [1000/2000], Loss: 0.5642
Epoch 1/30, Step [1500/2000], Loss: 0.5030
Epoch 1/30, Step [2000/2000], Loss: 0.3807
Accuracy on the test images: 66.25%
Accuracy on the train images: 65.8375%
Epoch 2/30, Step [500/2000], Loss: 0.6324
Epoch 2/30, Step [1000/2000], Loss: 0.4885
Epoch 2/30, Step [1500/2000], Loss: 0.9057
Epoch 2/30, Step [2000/2000], Loss: 0.6156
Accuracy on the test images: 69.4%
Accuracy on the train images: 71.2375%
Epoch 3/30, Step [500/2000], Loss: 0.6981
Epoch 3/30, Step [1000/2000], Loss: 0.6236
Epoch 3/30, Step [1500/2000], Loss: 0.7666
Epoch 3/30, Step [2000/2000], Loss: 0.5796
Accuracy on the test images: 72.75%
Accuracy on the train images: 72.5125%
Epoch 4/30, Step [500/2000], Loss: 0.5756
Epoch 4/30, Step [1000/2000], Loss: 0.9157
Epoch 4/30, Step [1500/2000], Loss: 0.5459
Epoch 4/30, Step [2000/2000], Loss: 0.7777
Accuracy on the test images: 73.2%
Accuracy on the train images: 74.275%
Epoch 5/30, Step [5

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(512, 2)
        self.convres = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        y = self.pool(self.pool(self.pool(self.convres(x))))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x += y
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [8]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [9]:
import time

st=time.time()

train_acc1=[]
test_acc1=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc1.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc1.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')

    with open('Model4_test.pkl','wb') as file:
            pickle.dump(test_acc1,file)
    with open('Model4_train.pkl','wb') as file:
            pickle.dump(train_acc1,file)
    with open('Model4_test.pkl','rb') as file:
            test_acc1=pickle.load(file)
    with open('Model4_train.pkl','rb') as file:
            train_acc1=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')

Epoch 1/30, Step [500/2000], Loss: 0.5384
Epoch 1/30, Step [1000/2000], Loss: 0.6657
Epoch 1/30, Step [1500/2000], Loss: 0.5247
Epoch 1/30, Step [2000/2000], Loss: 0.7046
Accuracy on the test images: 59.4%
Accuracy on the train images: 60.6625%
Epoch 2/30, Step [500/2000], Loss: 0.5271
Epoch 2/30, Step [1000/2000], Loss: 0.4468
Epoch 2/30, Step [1500/2000], Loss: 0.6901
Epoch 2/30, Step [2000/2000], Loss: 0.5911
Accuracy on the test images: 68.1%
Accuracy on the train images: 67.95%
Epoch 3/30, Step [500/2000], Loss: 0.5050
Epoch 3/30, Step [1000/2000], Loss: 0.2143
Epoch 3/30, Step [1500/2000], Loss: 0.7258
Epoch 3/30, Step [2000/2000], Loss: 0.7454
Accuracy on the test images: 69.95%
Accuracy on the train images: 70.275%
Epoch 4/30, Step [500/2000], Loss: 0.6105
Epoch 4/30, Step [1000/2000], Loss: 0.2710
Epoch 4/30, Step [1500/2000], Loss: 0.5116
Epoch 4/30, Step [2000/2000], Loss: 0.4573
Accuracy on the test images: 71.1%
Accuracy on the train images: 72.5875%
Epoch 5/30, Step [500/

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.dropout = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(512, 2)
        self.convres = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        y = self.pool(self.pool(self.pool(self.convres(x))))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x += y
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [11]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),momentum=0.9,lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [12]:
import time

st=time.time()

train_acc2=[]
test_acc2=[]

num_epochs = 30

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc2.append(100 * correct / total)
    print(f'Accuracy on the test images: {100 * correct / total}%')

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in train_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_acc2.append(100 * correct / total)
    print(f'Accuracy on the train images: {100 * correct / total}%')
    
    with open('Model5_test.pkl','wb') as file:
            pickle.dump(test_acc2,file)
    with open('Model5_train.pkl','wb') as file:
            pickle.dump(train_acc2,file)
    with open('Model5_test.pkl','rb') as file:
            test_acc2=pickle.load(file)
    with open('Model5_train.pkl','rb') as file:
            train_acc2=pickle.load(file)

ed=time.time()
print('Training finished in '+str(ed-st)+' Seconds!')

Epoch 1/30, Step [500/2000], Loss: 0.7633
Epoch 1/30, Step [1000/2000], Loss: 0.8540
Epoch 1/30, Step [1500/2000], Loss: 0.7273
Epoch 1/30, Step [2000/2000], Loss: 0.6738
Accuracy on the test images: 65.3%
Accuracy on the train images: 64.9625%
Epoch 2/30, Step [500/2000], Loss: 0.6766
Epoch 2/30, Step [1000/2000], Loss: 0.4619
Epoch 2/30, Step [1500/2000], Loss: 0.5125
Epoch 2/30, Step [2000/2000], Loss: 0.7479
Accuracy on the test images: 69.1%
Accuracy on the train images: 70.175%
Epoch 3/30, Step [500/2000], Loss: 0.3708
Epoch 3/30, Step [1000/2000], Loss: 0.1834
Epoch 3/30, Step [1500/2000], Loss: 0.8683
Epoch 3/30, Step [2000/2000], Loss: 0.3774
Accuracy on the test images: 67.45%
Accuracy on the train images: 69.275%
Epoch 4/30, Step [500/2000], Loss: 0.3179
Epoch 4/30, Step [1000/2000], Loss: 0.5109
Epoch 4/30, Step [1500/2000], Loss: 0.6973
Epoch 4/30, Step [2000/2000], Loss: 0.7660
Accuracy on the test images: 73.3%
Accuracy on the train images: 72.875%
Epoch 5/30, Step [500/